In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv("./模型訓練資料/tbrain_train_final_0610.csv", encoding = "utf8")
df.head(5)

,news_ID,hyperlink,content,name
0,1,https://news.cnyes.com/news/id/4352432,0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35...,[]
1,2,https://udn.com/news/story/120775/4112519,10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件 ### 省略內文 ### 北...,[]
2,3,https://www.mirrormedia.mg/story/20191008soc011/,2019.10.08 01:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從？文...,[]
3,4,https://www.chinatimes.com/realtimenews/201909...,58歲林姓男子昨凌晨與朋友聚餐結束後 ### 省略內文 ### 而地點就距離林家僅30公尺。...,[]
4,5,http://domestic.judicial.gov.tw/abbs/wkw/WHD9H...,例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家...,[]


In [3]:
df['website'] = df['hyperlink'].map(lambda x: x.split("/")[2]).tolist()
df['title'] = ''
df['content'] = ''

In [4]:
myparts = ['news.mingpao.com','www.bnext.com.tw','news.ltn.com.tw','finance.technews.tw','www.fsc.gov.tw','news.tvbs.com.tw','www.cw.com.tw','www.businesstoday.com.tw','sina.com.hk','www.ettoday.net','hk.on.cc']
df2 = df[df['website'].isin(myparts)]
df2 = df2.sort_values('website')
df2 = df2.reset_index(drop = True)

In [57]:
target = 'hk.on.cc'

In [58]:
## main program
exception_list = []
count = 0
# headers = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',
#     'Cookie': 'eyJpdiI6Ijd5VjVnU0ZIdnZ5TFJDUVRrR0picHc9PSIsInZhbHVlIjoiSHdIQ1pWYUZOazJcLzhIYWx2aHpEeDZudjE1UkhVR1NcLzVUYUo4OXNQSzMwR1R0QSs5bDEzeHpKc1B1aExXaDJ3IiwibWFjIjoiMmE5OTRiYjMwYjZiMmExNmE5MTE5MDZmMjg5MDJhOGNhYmJlMDdkYjc3ZWVlNzRmNjMxYmRmYmUxZWQzNjNmNiJ9',
# }
# session = requests.Session()
for i, row in df2[df2['website'] == target].iterrows():
    resp = requests.get(row['hyperlink'])
#     resp = requests.get(row['hyperlink'], headers=headers)
    resp.encoding = 'utf8'
    soup = BeautifulSoup(resp.text, "html.parser")
    if resp.status_code == 200:
        if(row['website'] == 'news.mingpao.com'): ##明報
            try:
                title = soup.find("div", id = "articleTop").find_next_siblings("hgroup")[0].find("h1").getText()
                content = soup.find("div", id = "upper").getText() + soup.find("div", id = "lower").getText()
            except:
                exception_list.append(i)
                continue
                
                     
        if(row['website'] == 'www.bnext.com.tw'): ##數位時代
            try:
                title = soup.find("h1", class_ = "article_title bitem_title").getText()
                content = soup.find("div", class_ = "article_summary").getText() + soup.find("article", class_ = "main_content").getText()
            except:
                #content = soup.find("article", class_ = "article_summary").getText() + soup.find("article", class_ = "main_content").getText()
                exception_list.append(i)
                continue
                
                
        if(row['website'] == 'news.ltn.com.tw'): ##自由時報
            try:
                title = soup.find("div", attrs = {"class": "whitecon", "itemprop": "articleBody"}).find("h1").getText()
                content = ""
                for node in soup.find("div", class_ = "text boxTitle boxText").find_all("p"):
                     content += node.getText()
            except:
                exception_list.append(i)
                continue

                
        if(row['website'] == 'finance.technews.tw'): ##財經新報
            try:
                title = soup.find("h1", class_ = "entry-title").find("a").getText()
                content = soup.find("div", class_ = "indent").getText()
            except:
                exception_list.append(i)
                continue
        

        if(row['website'] == 'www.fsc.gov.tw'): ##金融監督管理委員會
            try:
                title = soup.find("div", id = "maincontent").find_all("div")[0].getText()
                content = soup.find("div", id = "maincontent").find_all("div")[1].getText()
            except:
                exception_list.append(i)
                continue
                
                
        if(row['website'] == 'news.tvbs.com.tw'): ##TVBS NEWS
            try:
                title = soup.find("div", class_ = "title margin_b20").find("h1").getText()
                nodes = soup.find("div", class_ = "h7 margin_b20")
                for node in nodes.find_all("div", align="center"): node.decompose()
                content = nodes.getText()
            except:
                exception_list.append(i)
                continue                

                
        if(row['website'] == 'www.cw.com.tw'): ##天下雜誌(需訂閱)
            try:
                title = soup.find("h1", class_ = "my10").getText()
                content = soup.find("div", class_ = "preface").getText() +\
                            soup.find("div", class_ = "article__body pt-d-40 pt-m-30 px20").getText()
            except:
                exception_list.append(i)
                continue

        if(row['website'] == 'www.businesstoday.com.tw'): ##今周刊
            try:
                title = soup.find("h1", class_ = "article__maintitle").getText()
                content = soup.find("div", class_ = "cke_editable font__select-content").getText()
            except:
                exception_list.append(i)
                continue        
                
                
        if(row['website'] == 'sina.com.hk'): ##新浪香港
            try:
                title = soup.find("div", class_ = "news-title").getText()
                nodes = soup.find("div", attrs = {"id" : "news-main-body", "class" : "news-body"})
                nodes.find("script", type = "text/javascript").decompose()
                content = nodes.getText()
            except:
                exception_list.append(i)
                continue                

                
        if(row['website'] == 'www.ettoday.net'): ##ETtoday新聞雲
            try:
                if(row['hyperlink'].split("/")[3] == "amp"):
                    title = soup.find("header", class_ = "article-desc").getText()
                    content = soup.find("div", class_ = "content-container").getText()        
                if(row['hyperlink'].split("/")[3] == "news"):   
                    title = soup.find("h1", attrs = {"class" : "title", "itemprop" : "headline"}).getText()
                    content = soup.find("div", attrs = {"class" : "story", "itemprop" : "articleBody"}).getText()
            except:
                exception_list.append(i)
                continue                 
                
                
        if(row['website'] == 'hk.on.cc'): ##東網
            try:
                title = soup.find("div", class_ ="topHeadline").find("h1").getText()
                content = soup.find("div", class_ ="breakingNewsContent").getText() 
            except:
                exception_list.append(i)
                continue             
                
                
        df2.at[i, 'title'] = title
        df2.at[i, 'content'] = content
    else:
        df2.at[i, 'title'] = "error"
        df2.at[i, 'content'] = "error"
    count += 1
    if(count % 100) == 0:
        print("{} in progress... {}/{}".format(target, count, df2[df2['website'] == target].shape[0]))

In [59]:
# observing the exception
for link in df2.loc[exception_list, :]['hyperlink'].tolist(): print(link)
df2.loc[exception_list, :]

,news_ID,hyperlink,content,name,website,title


In [60]:
## display the result
pd.set_option('display.max_rows', None)
df2[df2['website'] == target]

,news_ID,hyperlink,content,name,website,title
56,3446,https://hk.on.cc/hk/bkn/cnt/cnnews/20191115/bk...,\n\r\n\t\t\t\t\t\t美國司法部長巴爾周三去信聯邦通訊委員會（FCC），指中國...,[],hk.on.cc,斥華為中興不可信 美稱很快達成貿易協議
57,4108,https://hk.on.cc/hk/bkn/cnt/cnnews/20191204/bk...,\n\r\n\t\t\t\t\t\t上海奉賢區一家模具材料公司周三（4日）下午1時多發生爆炸...,[],hk.on.cc,進行鋼板電焊切割致爆炸 一名工人當場死亡
58,1656,https://hk.on.cc/hk/bkn/cnt/cnnews/20191105/bk...,\n\r\n\t\t\t\t\t\t中加關係自華為太子女孟晚舟被捕一事後降至冰點，加拿大卑詩...,[],hk.on.cc,中加外交爭端不斷 僅29%加拿大人對華持正面看法
59,404,https://hk.on.cc/hk/bkn/cnt/cnnews/20191115/bk...,\n\r\n\t\t\t\t\t\t「2019年支線航空與國產支線飛機發展論壇」周四（14日...,[],hk.on.cc,應對運量增長 中國未來20年需補充7630架客機
60,2323,https://hk.on.cc/hk/bkn/cnt/cnnews/20191011/bk...,\n\r\n\t\t\t\t\t\t中澳關係持續緊張。澳洲內政部長達頓（Peter Dutt...,[],hk.on.cc,澳洲指控中國黑客入侵 華外交部：應摒棄冷戰思維
61,766,https://hk.on.cc/hk/bkn/cnt/cnnews/20191218/bk...,\n\r\n\t\t\t\t\t\t四川內江市資中縣周三（18日）上午8時14分，發生黎克特...,[],hk.on.cc,內江5.2級地震18人傷 逾780房間倒塌損毀
62,2913,https://hk.on.cc/hk/bkn/cnt/cnnews/20191120/bk...,\n\r\n\t\t\t\t\t\t台灣高雄鳳山區一個住宅單位發生雙屍命案，一名47歲的陳姓...,[],hk.on.cc,台婦與女兒離奇陳屍屋內 事前曾與丈夫爭吵
63,657,https://hk.on.cc/hk/bkn/cnt/cnnews/20191117/bk...,\n\r\n\t\t\t\t\t\t台灣雲林縣褒忠鄉一間土雞場上周六（16日）發現有雞隻感染...,[],hk.on.cc,褒忠鄉土雞場確診禽流感病例 逾1.3萬隻雞被撲殺
64,1630,https://hk.on.cc/hk/bkn/cnt/cnnews/20190923/bk...,\n\r\n\t\t\t\t\t\t中國科技巨企華為被指有意聯同中國移動，收購陷入財困的巴西...,[],hk.on.cc,華為發聲明 否認有意收購巴西電訊商
65,4659,https://hk.on.cc/hk/bkn/cnt/news/20191026/bkn-...,\n\r\n\t\t\t\t\t\t香港海關本周三(23日)於機場入境大堂，例行檢查2名從秘...,"['沈陳婕楹', '沈駿州']",hk.on.cc,台灣男女涉販運2300萬元液態可卡因 還押12月候訊


In [61]:
## validate the result manually
i = 78
print(df2.loc[i, 'hyperlink'])
print(df2.loc[i, 'title'])
print(df2.loc[i, 'content'])

https://hk.on.cc/hk/bkn/cnt/cnnews/20190921/bkn-20190921114420622-0921_00952_001.html
泰籍男逾期留台　藏身小食店日以繼夜改槍


						台灣的移民署近日接獲線報，指桃園市楊梅區楊新路一家泰式小食店，有外籍人士在出租套房內改裝槍枝，並吸食毒品等。經調查確認後，移民署聯同楊梅分局偵查隊共同前往查緝，當場拘捕泰國籍的潘姓男疑犯，並查扣改裝長槍1枝、少量安非他命、吸食器2組及改槍工具一批等證物。
						



						案情指，現年36歲的潘男持觀光簽證赴台，已逾期停留長達509天，期間他一直居住在涉事泰式小食店3樓。據潘供述，他透過上網自學改裝槍枝的相關技術，並購買電鑽等器材，利用出租套房掩人耳目，又自設小型試槍室，以測試改裝槍枝威力。由於日以繼夜地改槍，潘更購買安非他命吸食提神。他一開始辯稱從不吸毒，但最後從袋中被搜出安非他命一包。
						



						目前專案小組正在循線持續擴大追查槍枝的製程及成品流向，全案偵訊後，潘男依違反槍炮彈藥刀械管制條例及毒品危害防制條例，移送桃園地檢署偵辦。
						



In [62]:
# # program development
# for link in df2[df2['website'] == target]['hyperlink'].tolist()[0:3]:
#     print(link)
#     resp = requests.get(link)
#     resp.encoding = 'utf8'
#     soup = BeautifulSoup(resp.text, "html.parser")
#     title = soup.find("div", class_ ="topHeadline").find("h1").getText()
#     content = soup.find("div", class_ ="breakingNewsContent").getText()        
#     print(title)
#     print(content)

In [76]:
order = ['news_ID', 'hyperlink', 'name', 'website', 'title', 'content']
df2 = df2[order]
df2.to_csv("./Dragons.csv", index=False)